In [14]:
test_read = open('./data/book_data/book_test.txt', 'r')
test_lines = test_read.readlines()

user_his = {}

for line in test_lines:
    uid, iid, ts = line.split(',')
    uid, iid = int(uid), int(iid)
    # print(uid, iid, ts)
    # break
    if uid not in user_his:
        user_his[uid] = []
    user_his[uid].append(iid)


In [15]:
!pip install pandas

In [16]:
import pandas as pd
import numpy as np

uid_ls = [k for k,v in user_his.items()]
u_his_ls = [v for k,v in user_his.items()]

test_data = pd.DataFrame(
    data = zip(uid_ls, u_his_ls), columns=['user_id','user_history']
)

In [17]:
test_data.head()

,user_id,user_history
0,543301,"[1, 713, 482, 149, 145370, 226973]"
1,543302,"[263, 8401, 80, 921, 1132, 5999, 15156, 1290, ..."
2,543303,"[3695, 47970, 23644, 15754, 81064, 619]"
3,543304,"[964, 22461, 1400, 5289, 323, 14868, 46619, 65..."
4,543305,"[90399, 2860, 72894, 21401, 6075, 3572, 474]"


In [18]:
test_data['given_user_history'] = test_data['user_history'].apply(
    lambda x:
        x[:int(0.8*len(x))] 
)
test_data['predicting_items'] = test_data['user_history'].apply(
    lambda x:
        x[int(0.8*len(x)):] 
)

In [19]:
test_data.head()

,user_id,user_history,given_user_history,predicting_items
0,543301,"[1, 713, 482, 149, 145370, 226973]","[1, 713, 482, 149]","[145370, 226973]"
1,543302,"[263, 8401, 80, 921, 1132, 5999, 15156, 1290, ...","[263, 8401, 80, 921, 1132, 5999, 15156, 1290, ...","[825, 48282, 4959, 65858, 713, 72, 31, 17726, ..."
2,543303,"[3695, 47970, 23644, 15754, 81064, 619]","[3695, 47970, 23644, 15754]","[81064, 619]"
3,543304,"[964, 22461, 1400, 5289, 323, 14868, 46619, 65...","[964, 22461, 1400, 5289, 323, 14868, 46619, 65...","[22143, 5895, 12393, 12988, 71090, 3526, 17704..."
4,543305,"[90399, 2860, 72894, 21401, 6075, 3572, 474]","[90399, 2860, 72894, 21401, 6075]","[3572, 474]"


In [20]:
valid_read = open('./data/book_data/book_valid.txt', 'r')
valid_lines = valid_read.readlines()

user_his = {}

for line in valid_lines:
    uid, iid, ts = line.split(',')
    uid, iid = int(uid), int(iid)
    # print(uid, iid, ts)
    # break
    if uid not in user_his:
        user_his[uid] = []
    user_his[uid].append(iid)

uid_ls = [k for k,v in user_his.items()]
u_his_ls = [v for k,v in user_his.items()]

valid_data = pd.DataFrame(
    data = zip(uid_ls, u_his_ls), columns=['user_id','user_history']
)

valid_data['given_user_history'] = valid_data['user_history'].apply(
    lambda x:
        x[:int(0.8*len(x))] 
)
valid_data['predicting_items'] = valid_data['user_history'].apply(
    lambda x:
        x[int(0.8*len(x)):] 
)

In [21]:
valid_data.head()

,user_id,user_history,given_user_history,predicting_items
0,482934,"[15554, 29666, 123531, 7845, 10726, 48700]","[15554, 29666, 123531, 7845]","[10726, 48700]"
1,482935,"[11476, 10832, 172, 22948, 457, 9268]","[11476, 10832, 172, 22948]","[457, 9268]"
2,482936,"[5175, 14966, 23142, 32202, 32203, 19912, 4103...","[5175, 14966, 23142, 32202, 32203, 19912, 4103...","[57177, 22266, 61335, 126551]"
3,482937,"[101967, 121040, 121074, 137867, 107717]","[101967, 121040, 121074, 137867]",[107717]
4,482938,"[44, 585, 1308, 8477, 1077]","[44, 585, 1308, 8477]",[1077]


In [22]:
valid_data.drop(columns=['user_history']).to_csv(
    './dataset/validation.tsv', sep='\t', index=False
)
test_data.drop(columns=['user_history']).to_csv(
    './dataset/test.tsv', sep='\t', index=False
)

In [24]:
# Step 1: Load file and build user histories
user_his = {}

with open('./data/book_data/book_train.txt', 'r') as f:
    for line in f:
        uid, iid, ts = line.strip().split(',')
        uid, iid = int(uid), int(iid)

        if uid not in user_his:
            user_his[uid] = []
        user_his[uid].append(iid)

# Step 2: Build autoregressive training samples
records = []

for uid, items in user_his.items():
    for i in range(1, len(items)):
        records.append({
            "user_id": uid,
            "user_history": items[:i],
            "next_item": items[i]
        })

# Step 3: Create DataFrame
train_df = pd.DataFrame(records)

# Display to confirm
print(train_df.head())


   user_id                     user_history  next_item
0        0                          [17978]        901
1        0                     [17978, 901]      97224
2        0              [17978, 901, 97224]        774
3        0         [17978, 901, 97224, 774]      85757
4        0  [17978, 901, 97224, 774, 85757]      74974


In [25]:
train_df.to_csv(
    './dataset/training.tsv', sep='\t', index=False
)

In [3]:
import pandas as pd

df = pd.read_csv('./dataset/training.tsv', sep='\t')
print(df['user_id'].nunique())

482934


In [4]:
import pandas as pd

df = pd.read_csv('./dataset/validation.tsv', sep='\t')
print(df['user_id'].nunique())

60367


In [5]:
import pandas as pd

df = pd.read_csv('./dataset/test.tsv', sep='\t')
print(df['user_id'].nunique())

60367
